In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt



In [ ]:
! mv kaggle.json /root/.kaggle

In [ ]:
import kaggle

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [ ]:
!kaggle competitions download -c dogs-vs-cats

403 - Forbidden - Permission 'competitions.participate' was denied


In [ ]:
! unzip -q dogs-vs-cats.zip

In [ ]:
!unzip -q train.zip

In [ ]:
#Image Data Generator requires folder structure
# Requires you to have a folder structure (but only if you flow_from_directory)
# flow_from_dataframe : that your information is stored in a df
# What informations ? Images and their corresponding labels

# filename    | label
# dog.001.jpg | dog
# cat.007.jpg | cat


In [ ]:
#Creating df
data =pd.DataFrame(columns=['filename','label'])
#copying all files into data['filename']
data['filename']= os.listdir("train/")
data['label'] = None

In [ ]:
data.sample(5)

,filename,label
21196,cat.8739.jpg,None
23083,cat.8501.jpg,None
19287,dog.2231.jpg,None
8165,cat.11941.jpg,None
8071,dog.378.jpg,None


In [ ]:
data['filename'].str.split('.',expand=True)[0]

0        cat
1        dog
2        cat
3        dog
4        cat
        ... 
24995    cat
24996    cat
24997    cat
24998    cat
24999    cat
Name: 0, Length: 25000, dtype: object

In [ ]:
data['label']= data['filename'].str.split('.',expand = True)[0]

In [ ]:
data.sample(5)

,filename,label
1290,cat.318.jpg,cat
22756,dog.6898.jpg,dog
8167,dog.6276.jpg,dog
9159,dog.1134.jpg,dog
23283,cat.12065.jpg,cat


In [ ]:
idg = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.0 , validation_split=0.1)

In [ ]:
#Training data set
batch_size= 64
train_idg = idg.flow_from_dataframe(dataframe = data, directory="train/",
                                    x_col='filename',y_col='label',target_size=(300,300),
                                    batch_size= batch_size,subset = "training")

Found 22500 validated image filenames belonging to 2 classes.


In [ ]:
#validation
val_idg = idg.flow_from_dataframe(dataframe= data,directory = "train/",
                                  x_col='filename',y_col = 'label', target_size =(300,300),
                                  batch_size = batch_size, subset ='validation')

Found 2500 validated image filenames belonging to 2 classes.


In [ ]:
#model creation
model= tf.keras.models.Sequential()
#Input Layer
model.add(tf.keras.layers.Input(shape=(300,300,3),name ='InputLayer'))
#CNN Model
# 1st chunk:
#filter to obtain ROI
model.add(tf.keras.layers.Conv2D(filters= 32,kernel_size= (3,3), strides= (1,1),
                                 activation = tf.keras.activations.relu, padding='valid',
                                 name ='Conv1'))
#image_size = (298,298,32)
#MaxPooling : to reduce size
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2),name='Pool1'))
#image_size = (149,149,32)

# 2nd Chunk:
#filter
model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides=(1,1),
                                 activation = tf.keras.activations.relu,
                                 padding ='valid',name='Conv2'))
# image_size = (147,147,64)
# MaxPool
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2),name='Pool2'))
#image size = (73,73,64)    out_shape =math.floor(image-filter +stride +[padding if needed]/stride)

# 3rd Chunk:
#filter
model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides=(1,1),
                                 activation = tf.keras.activations.relu,
                                 padding ='valid',name='Conv3'))
# image_size = (71,71,64)
# MaxPool
model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2),name='Pool3'))
#image_size =(35,35,64)

# Flatten Layer
model.add(tf.keras.layers.Flatten(name ='FlattenLayer'))
#Hidden Layer
model.add(tf.keras.layers.Dense(units=64,activation = tf.keras.activations.relu,
                                name='HiddenLayer1'))
#Output Layer
model.add(tf.keras.layers.Dense(units=2,activation = tf.keras.activations.softmax,
                                name='Output'))

# Summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 298, 298, 32)      896       
                                                                 
 Pool1 (MaxPooling2D)        (None, 149, 149, 32)      0         
                                                                 
 Conv2 (Conv2D)              (None, 147, 147, 32)      9248      
                                                                 
 Pool2 (MaxPooling2D)        (None, 73, 73, 32)        0         
                                                                 
 Conv3 (Conv2D)              (None, 71, 71, 64)        18496     
                                                                 
 Pool3 (MaxPooling2D)        (None, 35, 35, 64)        0         
                                                                 
 FlattenLayer (Flatten)      (None, 78400)             0

In [ ]:
# Explanation of Filters :
# Num of filters = 32
# Depth of image = 3
# Value inside a filter = 3x3 = 9
# 32 * 9 * 3
# + bias
# (32*9*3) + 32


# Num of filters = 64
# Depth of image = 32
# Value inside the filters = 3x3 = 9
# 64 * 32 * 9 + 64



In [ ]:
#Compilation of Model
model.compile(optimizer = tf.keras.optimizers.SGD(),
              loss= tf.keras.losses.categorical_crossentropy,
              metrics= ['acc'])

In [ ]:
#Fit model
model.fit(train_idg,validation_data= val_idg, batch_size= batch_size,epochs=15)

In [ ]:
# GPU -
# Google Colab - Shares a GPU with us for FREE!

# Runtime -> Change Runtime Type --> Hardware (GPU)